In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import csv

In [2]:
# data from World Developement Indicator Dataset:
# https://datacatalog.worldbank.org/dataset/world-development-indicators

# get all country data: afghanistan to zimbabwe
df = pd.read_csv("WDIData.csv")[67257:]
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
67257,Afghanistan,AFG,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67258,Afghanistan,AFG,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67259,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,22.330000,24.080000,26.170000,27.990000,30.100000,32.44000,NaN,NaN,NaN,NaN
67260,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,43.222019,69.100000,70.153481,89.500000,71.500000,97.70000,97.700000,NaN,NaN,NaN
67261,Afghanistan,AFG,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,29.572881,60.849156,62.875693,86.500512,64.573354,97.09936,97.091973,NaN,NaN,NaN


In [3]:
indicator_names = df["Indicator Name"].unique()

# looking for usefull attributes 
for name in indicator_names:
    if "Life expectancy at birth" in name:
        print(name)

Life expectancy at birth, female (years)
Life expectancy at birth, male (years)
Life expectancy at birth, total (years)


In [4]:
usefull_attributes = ["Life expectancy at birth, total (years)", "GDP per capita (constant 2010 US$)", "Population, total"]

life_expectancy = df.where(df["Indicator Name"] == usefull_attributes[0])
population = df.where(df["Indicator Name"] == usefull_attributes[2])
gdp_p_cap = df.where(df["Indicator Name"] == usefull_attributes[1])

In [46]:
# drop nan and data 2018/2019
population=population.dropna(how="all")
population_filtered = population.drop(["2018","2019"],axis=1).reset_index()

life_expectancy=life_expectancy.dropna(how="all")
life_expectancy_filtered = life_expectancy.drop(["2018","2019"],axis=1).reset_index()

gdp_p_cap=gdp_p_cap.dropna(how="all")
gdp_p_cap_filtered=gdp_p_cap.drop(["2018","2019"],axis=1).reset_index()

In [47]:
# sanity check country numbers and year numbers
population_filtered.shape, life_expectancy_filtered.shape, gdp_p_cap_filtered.shape, population_filtered.columns[4:], len(population_filtered.columns[4:])

((217, 64),
 (217, 64),
 (217, 64),
 Index(['Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965',
        '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974',
        '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
        '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
        '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
        '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
        '2011', '2012', '2013', '2014', '2015', '2016', '2017', 'Unnamed: 64'],
       dtype='object'),
 60)

In [73]:
# remove all indicies with more than 58 nan's,
# i.e countries without any data of each specific attribute
to_remove_indicies = set()
ind_1 = life_expectancy_filtered.loc[life_expectancy_filtered.isnull().sum(1)>58].index
ind_2 = gdp_p_cap_filtered.loc[gdp_p_cap_filtered.isnull().sum(1)>58].index
ind_3 = population_filtered.loc[population_filtered.isnull().sum(1)>58].index
to_remove_indicies.update(ind_1.tolist(), ind_2.tolist(), ind_3.tolist())

pop_final = population_filtered.drop(list(to_remove_indicies))
gdp_p_final = gdp_p_cap_filtered.drop(list(to_remove_indicies))
life_exp_final = life_expectancy_filtered.drop(list(to_remove_indicies))

In [75]:
# write output csv files
pop_final.to_csv("population.csv")
life_exp_final.to_csv("life_expectancy.csv")
gdp_p_final.to_csv("gdp_p_cap.csv")